<a href="https://colab.research.google.com/github/GuyRobot/AINotesBook/blob/main/TextGenerateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
import tensorflow as tf
import numpy as np

import collections
import time
from urllib.request import urlretrieve

tf.compat.v1.enable_eager_execution()

In [ ]:
url = 'https://www.cs.cmu.edu/~spok/grimmtmp/'

dir_name = 'stories'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

def download_stories(filename):
    print("Downloading file: ", dir_name + os.sep + filename)
    if not os.path.exists(os.path.join(dir_name, filename)):
        filename, _ = urlretrieve(url + filename, dir_name+os.sep+filename)
    else:
        print("File %s already exits" % filename)
    return filename


filenames = [format(i, '03d') + '.txt' for i in range(1, 101)]

for fn in filenames:
    download_stories(fn)

In [ ]:
filenames = [format(i, '03d') + '.txt' for i in range(1, 101)]
dir_name = 'stories'

def read_data(filename):
    with open(filename) as f:
        data =  tf.compat.as_str(f.read())
        data = data.lower()
        data = list(data)
    return data

global documents
documents = []
num_files = 100
for i in range(num_files):
    print("processing file %s" % os.path.join(dir_name, filenames[i]))
    chars = read_data(os.path.join(dir_name, filenames[i]))

    # break into bigrams
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0, len(chars)-2, 2)]
    # Create document
    documents.append(two_grams)
    print("Data size (chars) (document %d) %d" % (i, len(two_grams)))
    print("Sample string %s\n" % (two_grams[:50]))

processing file stories/001.txt
Data size (chars) (document 0) 3667
Sample string ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' w', 'he', 'n ', 'wi', 'sh', 'in', 'g ', 'st', 'il', 'l ', 'he', 'lp', 'ed', ' o', 'ne', ', ', 'th', 'er', 'e ', 'li', 've', 'd ', 'a ', 'ki', 'ng', '\nw', 'ho', 'se', ' d', 'au', 'gh', 'te', 'rs', ' w', 'er', 'e ', 'al', 'l ', 'be', 'au', 'ti', 'fu', 'l,']

processing file stories/002.txt
Data size (chars) (document 1) 4928
Sample string ['ha', 'rd', ' b', 'y ', 'a ', 'gr', 'ea', 't ', 'fo', 're', 'st', ' d', 'we', 'lt', ' a', ' w', 'oo', 'd-', 'cu', 'tt', 'er', ' w', 'it', 'h ', 'hi', 's ', 'wi', 'fe', ', ', 'wh', 'o ', 'ha', 'd ', 'an', '\no', 'nl', 'y ', 'ch', 'il', 'd,', ' a', ' l', 'it', 'tl', 'e ', 'gi', 'rl', ' t', 'hr', 'ee']

processing file stories/003.txt
Data size (chars) (document 2) 9745
Sample string ['a ', 'ce', 'rt', 'ai', 'n ', 'fa', 'th', 'er', ' h', 'ad', ' t', 'wo', ' s', 'on', 's,', ' t', 'he', ' e', 'ld', 'er', ' o', 'f ', 'wh', 'om', ' w

In [ ]:
def build_dataset(documents):
    chars = []
    # list of lists
    data_list = []

    for d in documents:
        chars.extend(d)
    print('%d character found.' % len(chars))

    count = []
    # bigrams sorted by their frequency
    count.extend(collections.Counter(chars).most_common())

    # Create dict map word to id by given the current length of the dictionary
    # UNK is for two rare word
    dictionary = dict({'UNK': 0})
    for char, c in count:
        # Only add if its frequency is more than 10
        if c > 10:
            dictionary[char] = len(dictionary)
    unk_count = 0
    # replace word with id of word
    for d in documents:
        data = list()
        for char in d:
            # if word in dictionary use the id of word
            # otherwise use id of UNK
            if char in dictionary:
                index = dictionary[char]
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
        data_list.append(data)

    # dict map id to word
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data_list, count, dictionary, reverse_dictionary

data_list, count, dictionary, reverse_dictionary = build_dataset(documents)
print('Most common words (+UNK)', count[:5])
print('Least common words (+UNK)', count[-15:])
print('Sample data', data_list[0][:10])
print('Sample data', data_list[1][:10])
print('Vocabulary: ',len(dictionary))
vocabulary_size = len(dictionary)
del documents  # To reduce memory.

449177 character found.
Most common words (+UNK) [('e ', 15229), ('he', 15164), (' t', 13443), ('th', 13076), ('d ', 10687)]
Least common words (+UNK) [('bj', 1), ('ii', 1), ('i?', 1), ('z ', 1), ('c.', 1), ('"k', 1), ('pw', 1), ('f?', 1), (' z', 1), ('xq', 1), ('nm', 1), ('m?', 1), ('\t"', 1), ('\tw', 1), ('tz', 1)]
Sample data [15, 28, 86, 23, 3, 95, 74, 11, 2, 16]
Sample data [22, 156, 25, 37, 82, 185, 43, 9, 90, 19]
Vocabulary:  544


In [ ]:
datasets = [tf.data.Dataset.from_tensor_slices(data) for data in data_list]
datasets


[<DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter shapes: (), types: tf.int32>,
 <DatasetV1Adapter s

In [ ]:
for i in datasets[1].take(5):
    print(reverse_dictionary[i.numpy()])

ha
rd
 b
y 
a 


In [ ]:

seq_length = 64
# examples_per_epoch = vocabulary_size // (seq_length + 1)

datasets_sequences = [dataset.batch(seq_length + 1, drop_remainder=True) for dataset in datasets]

for item in datasets_sequences[1].take(5):
    for c in item.numpy():
        print(reverse_dictionary[c], end='')
    print("\n" + "-" * 24)


hard by a great forest dwelt a wood-cutter with his wife, who had an
only child, a little girl three years old.  they were so poor
------------------------
,
however, that they no longer had daily bread, and did not know how to
get food for her.  one morning the wood-cutter went out so
------------------------
rrowfully
to his work in the forest, and while he was cutting wood, suddenly
there stood before him a tall and beautiful woman wit
------------------------
h a crown of
shining stars on her head, who said to him 'i am the virgin mary,
mother of the child jesus. you are poor and needy, 
------------------------
bring your child
to me, i will take her with me and be her mother, and care for her.'
the wood-cutter obeyed, brought his child, a
------------------------


In [ ]:
# Map text to input and target (both input and target have
# the same seq_length but target is shifted to right one character)
def split_input_target(chunk):
    input_text = chunk[:-1] # take all except the last character
    target_text = chunk[1:] # take all except the first character

    return input_text, target_text

datasets = [datasets_sequence.map(split_input_target) for datasets_sequence in datasets_sequences]
datasets


[<DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int32, tf.int32)>,
 <DatasetV1Adapter shapes: ((64,), (64,)), types: (

In [ ]:
for input_exp, target_exp in datasets[0].take(1):
    print("Input data: ")
    for i in input_exp.numpy():
        print(reverse_dictionary[i], end='')
    print("\nTarget data: ")
    for t in target_exp.numpy():
        print(reverse_dictionary[t], end='')

Input data: 
in olden times when wishing still helped one, there lived a king
whose daughters were all beautiful, but the youngest was so bea
Target data: 
 olden times when wishing still helped one, there lived a king
whose daughters were all beautiful, but the youngest was so beaut

In [ ]:
BATCH_SIZE = 64
embedding_dim = 64

rnn_units = 1024


datasets = [dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(1) for dataset in datasets]

len(datasets)

100

In [ ]:
# class MyModel(tf.keras.Model):
#   def __init__(self, vocab_size, embedding_dim, rnn_units):
#     super().__init__(self)
#     self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#     self.gru = tf.keras.layers.GRU(rnn_units,
#                                    return_sequences=True,
#                                    return_state=True)
#     self.dense = tf.keras.layers.Dense(vocab_size)
#
#   def __call__(self, inputs, states=None, return_state=False, training=False):
#     x = inputs
#     x = self.embedding(x)
#     print(x)
#     if states is None:
#       states = self.gru.get_initial_state(x)
#     x, states = self.gru(x, initial_state=states, training=training)
#     x = self.dense(x, training=training)
#
#     if return_state:
#       return x, states
#     else:
#       return x

In [ ]:
# my_model = MyModel(vocab_size=vocabulary_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

In [ ]:

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(units=rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


model = build_model(vocabulary_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 64)            34816     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3345408   
_________________________________________________________________
dense (Dense)                (64, None, 544)           557600    
Total params: 3,937,824
Trainable params: 3,937,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
"""
    For each character the model looks up the
    embedding, runs the GRU one timestep with
    the embedding as input, and applies the dense
    layer to generate logits predicting the log-likelihood of the next character:
"""
for input_example_batch, target_example_batch in datasets[1].take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# my_model.summary()

(64, 64, 544) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 92, 312, 216, 251, 252,  28, 116, 181, 187, 304,  24, 220,  74,
       126, 393, 514, 468, 337, 477, 379,  90, 357, 539, 439,  95, 173,
       132, 431, 525, 243, 414, 171, 366, 429, 425,  72,  75,  30, 180,
       248, 289, 412, 325, 439, 341, 313, 284, 416, 358,  54,   1, 395,
       380, 213, 360,  16, 403,  56,   7, 144, 514, 389, 534, 393])

In [ ]:
def loss_sparse(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss_sparse(target_example_batch, example_batch_predictions)
example_batch_loss.numpy().mean()

6.299084

In [ ]:
optimizer = tf.train.AdamOptimizer()

def train_step(inp, target):
    with tf.GradientTape() as g:
        predictions = model(inp)
        loss = tf.reduce_mean(loss_sparse(target, predictions))

    gradients = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:


EPOCHS = 10
checkpoint_dir = './stories_checkpoint'

checkpoint_predix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_predix,
    save_weights_only=True
)

for epoch in range(EPOCHS):
    start = time.time()
    loss = 0
    # reset hidden state
    model.reset_states()

    for (di, dataset) in enumerate(datasets):

      for (batch_n, (inp, target)) in enumerate(dataset):
          loss = train_step(inp, target)

          if di % 10 == 0:
              print('Epoch {} Batch {} Loss {}'.format(epoch + 1, di, loss))

      if (epoch + 1) % 5 == 0:
          model.save_weights(checkpoint_predix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


model.save_weights(checkpoint_predix.format(epoch=epoch))


Epoch 1 Batch 40 Loss 2.9742891788482666
Epoch 1 Batch 50 Loss 2.8155484199523926
Epoch 1 Batch 60 Loss 2.902259111404419
Epoch 1 Batch 60 Loss 2.848997116088867
Epoch 1 Batch 90 Loss 2.756791114807129
Epoch 1 Loss 2.8786
Time taken for 1 epoch 37.87354111671448 sec

Epoch 2 Batch 40 Loss 2.8236641883850098
Epoch 2 Batch 50 Loss 2.680950164794922
Epoch 2 Batch 60 Loss 2.694181203842163
Epoch 2 Batch 60 Loss 2.6612865924835205
Epoch 2 Batch 90 Loss 2.621386766433716
Epoch 2 Loss 2.6785
Time taken for 1 epoch 37.910555839538574 sec

Epoch 3 Batch 40 Loss 2.681704044342041
Epoch 3 Batch 50 Loss 2.546788215637207
Epoch 3 Batch 60 Loss 2.5215697288513184
Epoch 3 Batch 60 Loss 2.493364095687866
Epoch 3 Batch 90 Loss 2.490720272064209
Epoch 3 Loss 2.4949
Time taken for 1 epoch 38.14566254615784 sec

Epoch 4 Batch 40 Loss 2.5244507789611816
Epoch 4 Batch 50 Loss 2.4008736610412598
Epoch 4 Batch 60 Loss 2.351067543029785
Epoch 4 Batch 60 Loss 2.326223611831665
Epoch 4 Batch 90 Loss 2.3433668613

In [ ]:
model = build_model(vocabulary_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             34816     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3345408   
_________________________________________________________________
dense_1 (Dense)              (1, None, 544)            557600    
Total params: 3,937,824
Trainable params: 3,937,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Vectorize
char2idx = {u:i for i, u in zip(dictionary.values(), dictionary.keys())}
idx2char = {i:u for i, u in zip(dictionary.values(), dictionary.keys())}

# Convert all character to int base on char2idx dict
text_as_int = np.array([i for data in data_list for i in data])
len(text_as_int)

449177

In [ ]:

def generate_text_bigram(model, start_string):
  # Evaluation step (generating text using the learned model)
  # Number of characters to generate
  num_generate = 1000
  two_grams = [''.join(start_string[ch_i:ch_i+2]) for ch_i in range(0, len(start_string)-2, 2)]
  two_grams_ids = []
  for i in two_grams:
    two_grams_ids.append(dictionary[i])
  # Converting our start string to numbers (vectorizing)
  input_eval = tf.convert_to_tensor(two_grams_ids)
  input_eval = tf.expand_dims(input_eval, 0)
  print(input_eval.shape)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



In [ ]:
print(generate_text_bigram(model, start_string=u"a certain father had two sons, the elder of whom was smart ands"))


(1, 31)
a certain father had two sons, the elder of whom was smart ands
sleds do the haverything so have the dwarf.  "ah," w, and tolk
you wh.'eve talking came home down by ead, it.  wine go cornered all m-f becmentur manger cornup off her right into sil of them, and that in through the copelived himself as,
las leat out
their mithering.  he likesy.  he like gicotilvy
tailor were just into the great her." then the old
places, want over they, and the lion put up but of his
hough form.  then he pulses, there my so the vainage for you, so, i mannge bate my mart." "yes and the herseUNKt's he mady
evily sparits, and in and
so strank earn
appeautiful
cusplesnd on his tanderelmbre, i do the wall,
and begged himself until the seven mouth and l-did whole world came back to he comminded he day
snow-white with them, they went youther about of fulf of her taillad, and she was to giest.  how fall lost his st if beforest.  in for whilind no lock for own, and a king always look lood time he lasked ba

In [ ]:
!zip -r 